In [1]:
from utils.helpers import get_league_urls,fetch_and_map_league, fetch_all_league_seasons, get_athlete_info, fetch_events_for_league_season, process_event, get_odds_data, get_stat_data

from db import add_to_db, League, Athlete, Card, Fight, Odds, StatisticsForFight
import httpx
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from typing import List, Dict, Tuple
import sqlite3

### Update League Info

In [2]:
league_urls = get_league_urls()
mappings = []
all_season_urls = []
with ThreadPoolExecutor(max_workers=20) as executor:
    futures = {executor.submit(fetch_and_map_league, url): url for url in league_urls}
    for future in as_completed(futures):
        url = futures[future]
        try:
            league_mapping, season_url = future.result()
            
            mappings.append(league_mapping)
            all_season_urls.append(season_url)
        except Exception as e:
            print(f"⚠️ Failed to fetch/map {url}: {e}")


In [3]:

conn = sqlite3.connect('data/mma.db')

cursor = conn.cursor()

league_ids = cursor.execute("SELECT id FROM leagues").fetchall()
league_ids = [row[0] for row in league_ids]

athlete_ids = cursor.execute("SELECT id FROM athletes").fetchall()
athlete_ids = [row[0] for row in athlete_ids]

card_ids = cursor.execute("SELECT league, event_id FROM cards").fetchall()

conn.close()

mappings_ids = [mapping['id'] for mapping in mappings]
new_league_ids = [id for id in mappings_ids if id not in league_ids]
new_mappings = [mapping for mapping in mappings if mapping['id'] in new_league_ids]
add_to_db(new_mappings, League)

Successfully added 0 out of 0 records to leagues


### Update Athlete Info

In [4]:
# ─── 1) Configuration ──────────────────────────────────────────────────────────
BASE_URL = "https://sports.core.api.espn.com/v2/sports/mma/athletes"
COMMON_PARAMS = {"lang": "en", "region": "us", "limit": 1000}
MAX_WORKERS = 10  # adjust up/down based on your bandwidth and the server’s rate limits

# ─── 2) Fetch page count ───────────────────────────────────────────────────────
resp = httpx.get(BASE_URL, params=COMMON_PARAMS)
resp.raise_for_status()
page_count = resp.json().get("pageCount", 0)
print(f"→ Detected {page_count} pages")

# ─── 3) Define a page-fetch helper ─────────────────────────────────────────────
def fetch_page(pg: int):
    params = {**COMMON_PARAMS, "page": pg}
    r = httpx.get(BASE_URL, params=params)
    r.raise_for_status()
    items = r.json().get("items", [])
    return [item.get("$ref") for item in items]

# ─── 4) Dispatch threads & collect results ─────────────────────────────────────
all_athlete_urls = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submit one future per page
    futures = {executor.submit(fetch_page, pg): pg for pg in range(1, page_count + 1)}
    for future in as_completed(futures):
        pg = futures[future]
        try:
            refs = future.result()
            all_athlete_urls.extend(refs)
        except Exception as e:
            print(f"❌ Page {pg} failed: {e!r}")

print(f"✔ Retrieved {len(all_athlete_urls)} athlete URLs")

→ Detected 37 pages
✔ Retrieved 36079 athlete URLs


In [5]:
all_athlete_ids = [int(athlete.split('?')[0].split('/')[-1]) for athlete in all_athlete_urls]
new_athlete_ids = [x for x in all_athlete_ids if x not in athlete_ids]
new_athlete_urls = [f"http://sports.core.api.espn.com/v2/sports/mma/athletes/{x}?lang=en&region=us" for x in new_athlete_ids]


with ThreadPoolExecutor(max_workers=25) as executor:
    futures = {executor.submit(get_athlete_info, url): url for url in new_athlete_urls}
    for future in as_completed(futures):
        url = futures[future]
        try:
            athlete_data = future.result()
            add_to_db(athlete_data, Athlete)
        except Exception as e:
            print(f"Error fetching athlete data for {url}: {e}")
    

### Update Card Info

In [6]:
league_seasons = fetch_all_league_seasons(all_season_urls)
def get_all_event_urls(
    league_seasons: List[Dict[str, List[str]]],
    max_workers: int = 20
) -> List[str]:
    """
    Given your league_seasons list, spawn threads to fetch each league/season.
    Returns one flat list of all event URLs.
    """
    # 1) Flatten to a list of (league, season) tuples
    tasks: List[Tuple[str,str]] = []
    for mapping in league_seasons:
        for league, seasons in mapping.items():
            for season in seasons:
                tasks.append((league, season))

    all_event_urls: List[str] = []
    # 2) Thread pool
    with ThreadPoolExecutor(
        max_workers=min(max_workers, len(tasks))
    ) as executor:
        future_to_task = {
            executor.submit(fetch_events_for_league_season, league, season): (league, season)
            for league, season in tasks
        }
        # 3) Collect results as they come in
        for future in as_completed(future_to_task):
            league, season = future_to_task[future]
            try:
                urls = future.result()
                all_event_urls.extend(urls)
            except Exception as exc:
                # Log but don’t kill the whole run
                print(f"⚠️ Error fetching {league} season {season}: {exc!r}")

    return all_event_urls


# Suppose league_seasons is defined already
event_urls = get_all_event_urls(league_seasons, max_workers=25)

all_event_ids = [(x.split('leagues/')[-1].split('/')[0], int(x.split('?')[0].split('/')[-1])) for x in event_urls]

new_card_ids = [x for x in all_event_ids if x not in card_ids]
new_card_urls = [f"http://sports.core.api.espn.com/v2/sports/mma/leagues/{x[0]}/events/{x[1]}?lang=en&region=us" for x in new_card_ids]



In [7]:
new_event_fight_ids = []
new_odds_urls = []
new_stat_urls = []
max_workers = 25

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(process_event, u): u for u in new_card_urls}
    for fut in as_completed(futures):
        url = futures[fut]
        try:
            card, evs = fut.result()
            if card:
                # card_mappings.append(card)
                add_to_db(card, Card)
            for ev in evs:
                try:
                    # attempt to append the mapped row
                    # event_mappings.append(ev)
                    new_odds_urls.append(ev['odds_url'])
                    new_event_fight_ids.append([ev['event_id'], ev['fight_id']])
                    new_stat_urls.append(ev['fighter_1_statistics'])
                    new_stat_urls.append(ev['fighter_2_statistics'])
                    add_to_db(ev, Fight)
                except Exception as e:
                    # if something goes wrong, print out the keys you care about
                    print(f"⚠️ Mapping error for event_id={ev.get('event_id')} "
                          f"league={ev.get('league')}: {e}")
        except Exception as e:
            print(f"⚠️ process_event failed for URL {url}: {e}")

⚠️ process_event failed for URL http://sports.core.api.espn.com/v2/sports/mma/leagues/ifl/events/400253543?lang=en&region=us: list index out of range
⚠️ process_event failed for URL http://sports.core.api.espn.com/v2/sports/mma/leagues/other/events/400254568?lang=en&region=us: list index out of range
⚠️ process_event failed for URL http://sports.core.api.espn.com/v2/sports/mma/leagues/bellator/events/400251967?lang=en&region=us: list index out of range
⚠️ process_event failed for URL http://sports.core.api.espn.com/v2/sports/mma/leagues/other/events/400254438?lang=en&region=us: list index out of range
⚠️ process_event failed for URL http://sports.core.api.espn.com/v2/sports/mma/leagues/other/events/400254573?lang=en&region=us: list index out of range
⚠️ process_event failed for URL http://sports.core.api.espn.com/v2/sports/mma/leagues/other/events/400254157?lang=en&region=us: list index out of range
⚠️ process_event failed for URL http://sports.core.api.espn.com/v2/sports/mma/leagues/o

In [8]:

with ThreadPoolExecutor(max_workers=25) as executor:
    futures = {executor.submit(get_odds_data, url): url for url in new_odds_urls}
    for future in as_completed(futures):
        url = futures[future]
        try:
            odds_data = future.result()
            add_to_db(odds_data, Odds)
        except Exception as e:
            print(f"Error fetching athlete data for {url}: {e}")

            

Error fetching athlete data for None: Invalid type for url.  Expected str or httpx.URL, got <class 'NoneType'>: None
Error fetching athlete data for None: Invalid type for url.  Expected str or httpx.URL, got <class 'NoneType'>: None
Error fetching athlete data for None: Invalid type for url.  Expected str or httpx.URL, got <class 'NoneType'>: None
Error fetching athlete data for None: Invalid type for url.  Expected str or httpx.URL, got <class 'NoneType'>: None
Error fetching athlete data for None: Invalid type for url.  Expected str or httpx.URL, got <class 'NoneType'>: None
Error fetching athlete data for None: Invalid type for url.  Expected str or httpx.URL, got <class 'NoneType'>: None
Error fetching athlete data for None: Invalid type for url.  Expected str or httpx.URL, got <class 'NoneType'>: None
Error fetching athlete data for None: Invalid type for url.  Expected str or httpx.URL, got <class 'NoneType'>: None
Error fetching athlete data for None: Invalid type for url.  Exp

In [9]:
new_stat_urls = [new_stat_urls for new_stat_urls in new_stat_urls if new_stat_urls is not None]


with ThreadPoolExecutor(max_workers=25) as executor:
    futures = {executor.submit(get_stat_data, url): url for url in new_stat_urls}
    for future in as_completed(futures):
        url = futures[future]
        try:
            stat_data = future.result()
            add_to_db(stat_data, StatisticsForFight)
        except Exception as e:
            print(f"Error fetching athlete data for {url}: {e}")

### Get Next UFC Event Info

In [10]:
from utils.helpers import process_next_event
import httpx

ufc_events_url = "https://sports.core.api.espn.com/v2/sports/mma/leagues/ufc/events/?lang=en&region=us"
ufc_event_response = httpx.get(ufc_events_url)
ufc_event_response.raise_for_status()
ufc_event_data = ufc_event_response.json()

next_ufc_event_url = ufc_event_data['items'][0]['$ref']


cards, evs = process_next_event(next_ufc_event_url)









In [11]:
evs

[{'league_event_id_fight_id': 'ufc_600053732_401780174',
  'fight_title': None,
  'boxscoreAvailable': True,
  'playByPlayAvailable': False,
  'summaryAvailable': True,
  'league': 'ufc',
  'event_id_fight_id': '600053732_401780174',
  'event_id': 600053732,
  'fight_id': '401780174',
  'odds_url': 'http://sports.core.api.espn.com/v2/sports/mma/leagues/ufc/events/600053732/competitions/401780174/odds?lang=en&region=us',
  'officials_url': None,
  'rounds_format': 3,
  'seconds_per_round': 300.0,
  'match_number': 13,
  'card_segment': 'Prelims',
  'weight_class_id': '1007',
  'weight_class': "Women's Flyweight",
  'end_round': 0,
  'end_time': '-',
  'result_display_name': None,
  'fighter_1_id': '4683395',
  'fighter_1_winner': False,
  'fighter_1_linescore': None,
  'fighter_1_statistics': 'http://sports.core.api.espn.com/v2/sports/mma/leagues/ufc/events/600053732/competitions/401780174/competitors/4683395/statistics?lang=en&region=us',
  'fighter_2_id': '4689227',
  'fighter_2_winne